In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('data/AA_Supervised_dataset.csv')
df.head()

In [ ]:
# opsplitting in X en y
X = df.drop('Ingeschreven', axis=1)
y = df['Ingeschreven']

In [ ]:
# opsplitting in train en test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=df['Ingeschreven'])
X_train.shape, X_test.shape, y_train.shape, y_test.shape